<a href="https://colab.research.google.com/github/nrohrbach/KeyFiguresElectricalInstallations/blob/main/KeyFiguresElectricalInstallations36kV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kennzahlen Geobasisdatensatz Elektrische Anlagen mit einer Nennspannung von über 36kV

---

Notebook um Kennzahlen des Geobasisdatensatzes "Elektrische Anlagen mit einer Nennspannung von über 36kV abzuleiten.


**Offene Fragen**

*   Sollen bei Leitungen die Erdungen auch gezählt werden oder nicht?
*   Hat Swissgrid auch Stationen der Netzebene 4?

**ToDo**

*   Kennzahlen Stationen nach Netzebene 2 und 4 unterteilen





In [ ]:
#pip install geopandas

In [ ]:
#pip install datetime

In [224]:
pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [225]:
import geopandas as gpd
import pandas as pd
import numpy as np
from datetime import datetime

In [148]:
url = "https://data.geo.admin.ch/ch.bfe.elektrische-anlagen_ueber_36/gpkg/2056/ElektrischeAnlagenNennspannungUeber36kV_V1.gpkg"
dfLeitung = gpd.read_file(url, driver="GPKG", layer='ElektrischeAnlagenNennspannungUeber36kV_V1_ElektrischeAnlagen_Leitung')
dfStation = gpd.read_file(url, driver="GPKG", layer='ElektrischeAnlagenNennspannungUeber36kV_V1_ElektrischeAnlagen_Station')
dfMast = gpd.read_file(url, driver="GPKG", layer='ElektrischeAnlagenNennspannungUeber36kV_V1_ElektrischeAnlagen_Mast')

## Kennzahlen Leitungen

In [218]:
# Länge der Leitungen berechnen und nach Spannung aufsummieren
dfLeitung["Laenge"] = dfLeitung.length
dfLeitungKPI = dfLeitung.groupby(['StromnetzTyp','Spannung','LeitungTyp'])['Laenge'].sum()
dfLeitungKPI = dfLeitungKPI.to_frame().reset_index()
dfLeitungKPI['Datum'] = datetime.today().strftime("%Y-%m-%d")

In [150]:
#Speichern
dfLeitungKPI.to_csv("Kennzahlen_Leitungen.csv", header=False, index=False, mode='a')

## Kennzahlen Masten

In [152]:
dfMastKPI = dfMast.groupby(['StromnetzTyp','MastTyp'])['xtf_id'].count().reset_index()
dfMastKPI = dfMastKPI.rename(columns={'xtf_id':'Anzahl'})
dfMastKPI['Datum'] = datetime.today().strftime("%Y-%m-%d")

In [153]:
#Speichern
dfMastKPI.to_csv("Kennzahlen_Masten.csv", header=True, index=False, mode='a')

## Kennzahlen Stationen

In [154]:
dfStationKPI = dfStation.groupby(['StromnetzTyp','StationTyp'])['xtf_id'].count().reset_index()
dfStationKPI = dfStationKPI.rename(columns={'xtf_id':'Anzahl'})
dfStationKPI['Datum'] = datetime.today().strftime("%Y-%m-%d")

In [155]:
#Speichern
dfStationKPI.to_csv("Kennzahlen_Stationen.csv", header=True, index=False, mode='a')

## Kennzahlen Werke

In [156]:
# Leitungslänge pro Werk ableiten.
dfWerkeLeitung = dfLeitung
dfWerkeLeitung['Werk'] = dfWerkeLeitung['xtf_id'].str.split('_').str[0]
dfWerkeLeitung = dfWerkeLeitung.groupby(['Werk'])['Laenge'].sum()
dfWerkeLeitung = dfWerkeLeitung.to_frame().reset_index()
dfWerkeLeitung = dfWerkeLeitung.rename(columns={'Laenge':'LaengeLeitung'})

In [157]:
# Anzahl Masten pro Werk ableiten.
dfWerkeMast = dfMast
dfWerkeMast['Werk'] = dfWerkeMast['xtf_id'].str.split('_').str[0]
dfWerkeMast = dfWerkeMast.groupby(['Werk'])['xtf_id'].count().reset_index()
dfWerkeMast = dfWerkeMast.rename(columns={'xtf_id':'AnzahlMasten'})

In [159]:
# Anzahl Stationen pro Werk ableiten.
dfWerkeStation = dfStation
dfWerkeStation['Werk'] = dfWerkeStation['xtf_id'].str.split('_').str[0]
dfWerkeStation = dfWerkeStation.groupby(['Werk'])['xtf_id'].count().reset_index()
dfWerkeStation = dfWerkeStation.rename(columns={'xtf_id':'AnzahlStationen'})

In [160]:
# Zusammenführen
dfWerke = pd.merge(dfWerkeLeitung, dfWerkeMast, how="left", on=["Werk"])
dfWerke = pd.merge(dfWerke, dfWerkeStation, how="left", on=["Werk"])
dfWerke = dfWerke.fillna(0)
dfWerke['Datum'] = datetime.today().strftime("%Y-%m-%d")

In [161]:
#Speichern
dfWerke.to_csv("Kennzahlen_Werke.csv", header=True, index=False, mode='a')

## Kennzahlen für Visualisierung in Storymap ableiten

In [236]:
dfLeitungStorymap = dfLeitungKPI

# Gewünschte Kategorien definieren
Bedingungen = [dfLeitungStorymap['Spannung']=='S380kV',
               dfLeitungStorymap['Spannung']=='S220kV',
               (dfLeitungStorymap['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungStorymap['Spannung'].isin(['S150kV','S132kV','S130kV','S125kV','S110kV'])),
               (dfLeitungStorymap['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungStorymap['Spannung'].isin(['S66kV','S65kV','S60kV','S50kV','S36kV','Andere'])),
               (dfLeitungStorymap['StromnetzTyp']=='Bahnstrom') & (dfLeitungStorymap['Spannung']=='S132kV'),
               (dfLeitungStorymap['StromnetzTyp']=='Bahnstrom') & (dfLeitungStorymap['Spannung'].isin(['S66kV','Andere']))        
               ]

Spannungen = ['s380','s220','s150','s66','s132B','s66B']



In [281]:
 dfLeitungStorymap['Kategorie'] =  np.select(Bedingungen,Spannungen)
 dfLeitungStorymap

,StromnetzTyp,Spannung,LeitungTyp,Laenge,Datum,Kategorie
0,Allgemeinstrom,Andere,Freileitung,3.785600e+06,2023-05-03,s66
1,Allgemeinstrom,Andere,Kabelleitung,2.215100e+04,2023-05-03,s66
2,Allgemeinstrom,S110kV,Freileitung,1.390137e+06,2023-05-03,s150
3,Allgemeinstrom,S110kV,Kabelleitung,4.018931e+05,2023-05-03,s150
4,Allgemeinstrom,S132kV,Kabelleitung,9.624992e+04,2023-05-03,s150
5,Allgemeinstrom,S150kV,Freileitung,6.401418e+04,2023-05-03,s150
6,Allgemeinstrom,S150kV,Kabelleitung,3.967614e+05,2023-05-03,s150
7,Allgemeinstrom,S220kV,Freileitung,4.832284e+06,2023-05-03,s220
8,Allgemeinstrom,S220kV,Kabelleitung,2.501321e+04,2023-05-03,s220
9,Allgemeinstrom,S380kV,Freileitung,1.893143e+06,2023-05-03,s380


In [289]:
dfTotal

Kategorie,index,s150,s220,s380,s66
0,Laenge,2.349056e+06,4.857297e+06,1.915187e+06,4.915281e+06


In [305]:
dfTotal = dfLeitungStorymap.groupby('Kategorie').sum(numeric_only=True).reset_index()
dfTotal['Kategorie'] = 'Total_'+ dfTotal['Kategorie']
dfTotal = dfTotal.transpose()

dfTotal.columns = dfTotal.iloc[0]
dfTotal = dfTotal[1:]



dfTotal

Kategorie,Total_s150,Total_s220,Total_s380,Total_s66
Laenge,2349055.72573,4857296.789805,1915187.111824,4915281.230717


In [268]:
dfKabel = dfLeitungStorymap.loc[dfLeitungStorymap['LeitungTyp']=='Kabelleitung'].groupby('Kategorie').sum(numeric_only=True).reset_index
dfKabel

,Kategorie,Laenge
0,s150,894904.391234
1,s220,25013.209003
2,s380,22044.578551
3,s66,742579.917095


In [256]:
'Kabel_' + dfKabel['Kategorie']

0    Kabel_s150
1    Kabel_s220
2    Kabel_s380
3     Kabel_s66
Name: Kategorie, dtype: object

In [219]:
dfLeitungStorymap = dfLeitungKPI
dfLeitungStorymap['Kategorie'] = 'tbd'
dfLeitungStorymap

,StromnetzTyp,Spannung,LeitungTyp,Laenge,Datum,Kategorie
0,Allgemeinstrom,Andere,Freileitung,3.785600e+06,2023-05-03,tbd
1,Allgemeinstrom,Andere,Kabelleitung,2.215100e+04,2023-05-03,tbd
2,Allgemeinstrom,S110kV,Freileitung,1.390137e+06,2023-05-03,tbd
3,Allgemeinstrom,S110kV,Kabelleitung,4.018931e+05,2023-05-03,tbd
4,Allgemeinstrom,S132kV,Kabelleitung,9.624992e+04,2023-05-03,tbd
5,Allgemeinstrom,S150kV,Freileitung,6.401418e+04,2023-05-03,tbd
6,Allgemeinstrom,S150kV,Kabelleitung,3.967614e+05,2023-05-03,tbd
7,Allgemeinstrom,S220kV,Freileitung,4.832284e+06,2023-05-03,tbd
8,Allgemeinstrom,S220kV,Kabelleitung,2.501321e+04,2023-05-03,tbd
9,Allgemeinstrom,S380kV,Freileitung,1.893143e+06,2023-05-03,tbd


In [223]:
for Spannung in dfLeitungStorymap['Spannung']:
  if Spannung == 'S380kV':
    Kategorie = 'y'
    #print(Spannung)
  else:
    Kategorie = 'x'
    #print('x')


Kategorie

#dfLeitungStorymap

'x'

In [208]:
dfLeitungStorymap.loc()

,StromnetzTyp,Spannung,LeitungTyp,Laenge,Datum,Kategorie
0,Allgemeinstrom,Andere,Freileitung,3.785600e+06,2023-05-03,s380
1,Allgemeinstrom,Andere,Kabelleitung,2.215100e+04,2023-05-03,s380
2,Allgemeinstrom,S110kV,Freileitung,1.390137e+06,2023-05-03,s380
3,Allgemeinstrom,S110kV,Kabelleitung,4.018931e+05,2023-05-03,s380
4,Allgemeinstrom,S132kV,Kabelleitung,9.624992e+04,2023-05-03,s380
5,Allgemeinstrom,S150kV,Freileitung,6.401418e+04,2023-05-03,s380
6,Allgemeinstrom,S150kV,Kabelleitung,3.967614e+05,2023-05-03,s380
7,Allgemeinstrom,S220kV,Freileitung,4.832284e+06,2023-05-03,s380
8,Allgemeinstrom,S220kV,Kabelleitung,2.501321e+04,2023-05-03,s380
9,Allgemeinstrom,S380kV,Freileitung,1.893143e+06,2023-05-03,s380


In [171]:
df380 = dfLeitungKPI.loc[dfLeitungKPI['Spannung']=='S380kV']
df220 = dfLeitungKPI.loc[dfLeitungKPI['Spannung']=='S220kV']
df150 = dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung'].isin(['S150kV','S132kV','S130kV','S125kV','S110kV']))]
df66 = dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung'].isin(['S66kV','S65kV','S60kV','S50kV','S36kV','Andere']))]
df132B = dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Bahnstrom') & (dfLeitungKPI['Spannung']=='S132kV')]
df66B = dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Bahnstrom') & (dfLeitungKPI['Spannung'].isin(['S66kV','Andere']))]

In [ ]:
dfLeitungStorymap['Kategorie'] = 

In [ ]:
Spannungen = ['380','220','150','66','132B','66B']

In [144]:
dfLeitungStorymap = pd.DataFrame()

In [138]:
df220
df380

,StromnetzTyp,Spannung,LeitungTyp,Laenge,Datum
9,Allgemeinstrom,S380kV,Freileitung,1.893143e+06,2023-05-03
10,Allgemeinstrom,S380kV,Kabelleitung,2.204458e+04,2023-05-03


In [146]:
s380kV_Kabel = df380.loc[df380['LeitungTyp']=='Kabelleitung']['Laenge']
s380kV_Total = df380['Laenge'].sum()
s220kV_Kabel = df220.loc[df220['LeitungTyp']=='Kabelleitung']['Laenge']
s220kV_Total = df220['Laenge'].sum()


4857296.789804842

In [141]:
dfLeitungStorymap

,380kV_Kabel,380kV_Total,220kV_Kabel,220kV_Total
10,22044.578551,1.915187e+06,NaN,4.857297e+06


In [60]:
dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung'].isin(['S150kV','S132kV','S130kV','S125kV','S110kV']))]['Laenge'].sum()

2349055.7257298105

In [29]:
dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung']=='S380kV')]['Laenge'].sum()

1915187.111823888

In [ ]:
dfLeitungStorymap['150kV_Kabel'] = dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung'].isin(['S150kV','S132kV','S130kV','S125kV','S110kV'])) & (dfLeitungKPI['LeitungTyp']=='Kabelleitung')]['Laenge']

In [167]:
dfLeitungStorymap = pd.DataFrame()

In [168]:
dfLeitungStorymap['220kV_Kabel'] = dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung']=='S220kV') & (dfLeitungKPI['LeitungTyp']=='Kabelleitung')]['Laenge']
dfLeitungStorymap['380kV_Kabel'] = dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung']=='S380kV') & (dfLeitungKPI['LeitungTyp']=='Kabelleitung')]['Laenge']


dfLeitungStorymap['220kV_Total'] = dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung']=='S220kV')]['Laenge'].sum()
dfLeitungStorymap['380kV_Total'] = dfLeitungKPI.loc[(dfLeitungKPI['StromnetzTyp']=='Allgemeinstrom') & (dfLeitungKPI['Spannung']=='S380kV')]['Laenge'].sum()

dfLeitungStorymap

,220kV_Kabel,380kV_Kabel,220kV_Total,380kV_Total
8,25013.209003,NaN,4.857297e+06,1.915187e+06
